### This version of Task 1 formulates the task of finding minimum as QUBO and solves it using QAOA
#### Given an array of numbers $N = [n_0, n_1, ..., n_D]$, encode index of each number in the array as in one hot encoding, e.g. <br> 
$n_0 \Rightarrow [1, 0, 0, ... , 0]$, $n_1 \Rightarrow [0, 1, 0, 0 ... 0]$, ... , $ n_N \Rightarrow [0, 0, ... , 1]$<br>
Define Hamiltonian over binary variable x as $H(N, x) = \left( \sum_{n=0}^{D} N_i x_i \right)^2$, this way the minimum x will correspond to the encoded index of the smallest element $n_j$.<br>
Also, one has to add the constraint, which limits all the binary x's only to one-hot encoded solutions, where $\sum x_i = 1$<br>
For this, one adds penalty $P(x) = \left(\sum_i x_i - 1 \right)^2 = -\sum x_i + \sum_{i<j} x_i x_j$<br>
Total cost is the Hamiltonian plus penalty $Q(N,x) = H(N,x) + \lambda P(x)$<br>
We formulate this optimization as QuadraticProgram from Qiskit, and solve it using QAOA.

In [177]:
from qiskit.utils import algorithm_globals
from qiskit.algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [178]:
# Returns linear and quadratic coefficients of H for QuadraticProgram
def hamiltonian(n: list) -> (list, dict):
    linear = [ni**2 for ni in n]
    xs = [("x",i) for i in range(len(n))]
    nxs = list(zip(xs,n))
    quadratic = {(x+str(i),x+str(j)) : ni*nj for ((x,i), ni) in nxs for ((x,j),nj) in nxs  if i<j}
    return (linear, quadratic)

In [179]:
hamiltonian([17,3,49])

([289, 9, 2401], {('x0', 'x1'): 51, ('x0', 'x2'): 833, ('x1', 'x2'): 147})

In [180]:
# Returns linear and quadratic coefficients of P for QuadraticProgram (here lambda above is l)
def penalty(l: float, n: list) -> (list, dict):
    linear = [-l for ni in n]
    quadratic = {("x"+str(i), "x"+str(j)) : l for i in range(len(n)) for j in range(len(n)) if i<j}
    return(linear, quadratic)

In [181]:
penalty(4761,[17,3,49])

([-4761, -4761, -4761],
 {('x0', 'x1'): 4761, ('x0', 'x2'): 4761, ('x1', 'x2'): 4761})

In [182]:
# Combines coefficients of Hamiltonian and penalty with lambda = (sum of n's)^2
def qubo_params(n: list) -> (list, dict):
    l = sum(n)**2
    (h_lin, h_q) = hamiltonian(n)
    (p_lin, p_q) = penalty(l, n)
    q_lin = [x + y for (x,y) in zip(h_lin,p_lin)]
    q_q = {k: h_q[k] + p_q[k] for k in h_q.keys()}
    return (q_lin, q_q)

In [183]:
qubo_params([17,3,49])

([-4472, -4752, -2360],
 {('x0', 'x1'): 4812, ('x0', 'x2'): 5594, ('x1', 'x2'): 4908})

In [184]:
""" Formulates QuadraticProgram using qubo_params above and solves for minimum using QAOA.
    Then finds maximum
"""
def find_the_largest_number(number_1: int, number_2: int) -> int:
    numbers = [number_1, number_2]
    qubo = QuadraticProgram()
    qubo.binary_var("x0")
    qubo.binary_var("x1")
    (lin,quadr) = qubo_params(numbers)
    qubo.minimize(linear=lin, quadratic=quadr)
    qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0.0, 0.0])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    qaoa_result = qaoa.solve(qubo)
    indicator = list(qaoa_result.x)
    return numbers[indicator.index(0)]

In [185]:
find_the_largest_number(9,7)

9